#  3주차 중앙일보 인권기사 크롤링



# Import library
*  BeautifulSoup는 HTML과 XML 파일로부터 데이터를 뽑아내기 위한 파이썬 라이브러리이다  
*  urllib HTML에 연결하기 위한 라이브러리  
*  pandas dataframe을 사용하기 위한 라이브러리
*  import를 통해 모듈 전체를 불러올 수도 있고, from...import 방식을 써서 한 모듈에서 일정 함수만을 불러서 쓸 수도 있음. 

In [4]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd
import re #정규식을 쓰기위해서 필요한 패키지
pd.set_option('display.max_colwidth', -1)

<ipython-input-4-9ea9960b25f5>:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Global variable (전역변수) 
*  전역 변수는 어느 지역에서나 사용할 수 있는 변수입니다.  
*  전역 변수는 모두 대문자로 표기하였습니다.  
https://wikidocs.net/18
*  전역변수는 스크립트에 최상단에 선언해야 함. 

# Local variable (지역변수)
*  함수 내부에 선언된 변수로써 함수 내에서 만 유효한 변수임.  

---

**TARGET_URL_KEYWORD:** 신문사 키워드  
**TARGET_URL_DATA:** 날짜 
*   StartSearchDate : 시작날짜
*   EndSearchDate : 끝나는날짜  

**TARGET_URL_REST:** 
*   SortType : New=최신순 Accuracy=정확도 
*   SearchCategoryType : JoongangNews=중앙뉴스 TotalNews=전체
*   PeriodType : all=전체 Oneday=1일 OneWeek=1주 OneMonth=1달 

---
# List (리스트)
리스트는 여러 개의 데이터들을 하나의 변수로 관리하기 위해 사용합니다.  
https://wikidocs.net/14   
**TITLE_OF_ARTICLE**: 기사 제목을 저장할 리스트  
**CONTENT_OF_ARTICLE**: 기사 내용을 저장할 리스트  
**DATE_OF_ARTICLE**: 기사 날짜를 저장할 리스트  


In [26]:
#TARGET_URL_KEYWORD = 'https://news.joins.com/Search/JoongangNews?page=&Keyword=%EC%9D%B8%EA%B6%8C&'
#TARGET_URL_DATE = 'PeriodType=DirectInput&StartSearchDate=01%2F01%2F2020%2000%3A00%3A00&EndSearchDate=05%2F22%2F2020%2000%3A00%3A00&'
#TARGET_URL_REST = 'SortType=New&SearchCategoryType=JoongangNews'


#TARGET_URL = TARGET_URL_KEYWORD + 
TARGET_URL_DATE + TARGET_URL_REST # 예전날짜 2020년 1월 1일-5월 22일
# 아래가 원래의 URL이었는데
# https://news.joins.com/Search/JoongangNews?Keyword=%EC%9D%B8%EA%B6%8C&StartSearchDate=01%2F01%2F2020%2000%3A00%3A00&EndSearchDate=05%2F22%2F2020%2000%3A00%3A00&SortType=New&SearchCategoryType=JoongangNews&PeriodType=DirectInput&ScopeType=All&ImageType=All&JplusType=All&BlogType=All&ImageSearchType=Image&TotalCount=0&StartCount=0&IsChosung=False&IssueCategoryType=All&IsDuplicate=True&Page=&PageSize=10&IsNeedTotalCount=True
# 페이지를 옮겨가면, URL이 바뀌는 현상....새로 뭔가 바뀌었음....
#https://news.joins.com/Search/JoongangNews?page=&Keyword=%EC%9D%B8%EA%B6%8C&PeriodType=DirectInput&StartSearchDate=01%2F01%2F2020%2000%3A00%3A00&EndSearchDate=05%2F22%2F2020%2000%3A00%3A00&SortType=New&SearchCategoryType=JoongangNews


# 2021년 1월 1일부터 3월 5일까지 넣어 봤음
TARGET_URL = 'https://news.joins.com/Search/totalnews?page=&Keyword=%EC%9D%B8%EA%B6%8C&PeriodType=DirectInput&StartSearchDate=01%2F01%2F2021%2000%3A00%3A00&EndSearchDate=03%2F05%2F2021%2000%3A00%3A00&SortType=New&SourceGroupType=Joongang&SearchCategoryType=TotalNews'

TITLE_OF_ARTICLE = []
CONTENT_OF_ARTICLE = []
DATE_OF_ARTICLE = []

#  Function (함수)
*  함수는 입력값을 가지고 어떤 일을 수행한 다음에 그 결과물을 내어놓는 것입니다.  
https://wikidocs.net/24
*  함수를 정의한 후, 그 함수에 삽입할 값을 넣으면 결과값이 나오는 구조임
*  아래 def get_link_from_news_title(page_num, URL): 에서 page_num, URL 은 미지수로써 향후 사용될 값을 담는 변수를 의미. 
*  반환값 (return)은 함수 수행의 결과값을 의미. 


# Crawling article 
**get_link_from_news_title** : 기사 검색 페이지에서 기사 제목에 링크된 기사의 주소를 가져온다.
*   current_page_num : 1페이지 = 1, 2페이지 = 2 1씩 증가하므로 1 + i
*   position : current_page_num을 URL에 삽입할 위치를 찾는다.
*   URL_with_page_num : current_page_num을 URL에 삽입한다.
*   source_code_from_URL : 크룰링을 하기 위해 지정된 URL에 연결을 요청
*   soup : 데이터를 가져오기 위한 BeautifulSoup Library
*   for title in soup.find_all : 한 페이지에 10개의 타이틀이 존재하는데 순서대로 기사 타이틀에 접근한다.
*   title_link : URL이 위치하는 곳을 가리킨다.
*   TITLE_OF_ARTICLE.append  : 기사 제목을 리스트에 넣는다.
*   article_URL : 기사 본문이 위치하는 URL을 가져온다.
*   get_text : 기사의 내용을 가져오기 위한 함수  
https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* page index를 잡는 것은 현재처럼 'page='을 해도 되고, 아애 좁혀서 'p'로 해도 됨. 신문사에서 포지션을 p 한자에 잡아 놓은 것으로 보임(그러나 이 경우에는 URL에 또 다른 p가 있을 수도 있음으로 신중할 필요). 이렇게 바꿔도-다른 p가 없다면- 코드는 잘돌아감. URL을 완성하는 'URL_with_page_num'의 [:postion_5]에서 +5의 의미는 p를 기준으로 뒤 다섯번째 칸까지 읽으라는 것임 (0부터 카운트하면 &가 5가 됨. 그런데 여기서 5는 포함하지 않음). 문자열(*문자1개 혹은 복수로 이뤄진 단위)은 주어진 개별 주소를 통해 내부의 요소를 다양하게 추출할 수 있음. 인덱싱(indexing)을 통해 원하는 문자 각각을 추출하는 것임. 반면, TARGET_URL[position+5:]의 경우는 0부터 카운트 하여 &가 5가 되는데, 이때는 5에 해당되는 &포함이 됨. 이렇게 되어서 URL이 모두 잘 연결이 되고, 원하는 자리에 페이지 번호를 넣어줄 수 있게 됨. 
* title_link[0] 와 같이 [0]이 들어가는 이유는 중앙일보에서 list 형식으로 해당 코드를 제공했기 때문임. String형식으로 주면 [0]을 붙힐 필요가 없음. 아래의 코드에서 중앙에서 애초에 'a'태그를 list로 주었기 때문에, 그 다음행의 title_link에 [0]을 붙힌 것임.
* 리스트 데이터는 [ ]로 표기하며, 문자열 데이터와 유사하게 처리됨. [ ] 안에 있는 0, 1 등은 리스트 내의 "개별 주소값"을 의미한다.
title_link[0].string #'a'태그의 0번째 list에서 string으로된 문자열을 가져온다는 의미임.

* 신문사에 따라 list로 주느냐 string으로 주느냐에 차이가 나기 때문에 사실 이를 구별하기가 매우 어려움. 결국 [0]이나 [1]넣어보고 또 빼고 해보면서 스스로가 경험적으로 터득해야 하는 부분임!
* def get_link_from_news_title(page_num, URL)....이 부분에서 page_num과 URL과 같이 새로 만들려고 하는 함수 안에 있는 변수들은 정의가 안되어 있는 상태임. 이런 애들을 가져다 쓰겠다하고 이름만 넣어주는 것임. 주욱 내려가서 마지막 메인함수에 있는 두개의 변수. 즉 # get_link_from_news_title(page_num, TARGET_URL) ...에서 page_num과 TARGET_URL을 정의가 안되어있는 윗 부분의 page_num과 URL에 넣어주는 구조임. 이렇게 보면 각각의 노트가 별개가 아니라 블럭처럼 연결되어서 개발자가 원하는 작업을 '협업하면서' 수행한다고 보면됨.
* TARGET_URL을 위에서 변수로 만들었는데, def 코드에서 왜 URL이 또 들어가느냐? 그건 주욱 내려와서 매인함수와 상관이 있음. 매인함수에서 TARGET_URL을 넣어주게 되고, 이것이 앞서 우리가 만든 함수(get_link_from_news_title)의 URL로 연결되는 것임. 이 연결고리가 없으면 만든 함수가 실행이 되지 않음.

#  웹페이지의 구조 
** '정치학을 위한 파이썬'71-77쪽 참고'
*  XML 문서는 기본적으로 HTML 문서와 유사한 구조를 지니고 있음 (트리구조, 태그와 태그 사이에 데이터가 놓이는 것, 여는태그-닫는태그, 페이지 소스보기 및 검사)
*  HTML의 기본구조...전체를 아우르는 <html>...</html>, 페이지를 정의하는 <head>...</head>, 본문 내용을 담고 있는 <body>...</body> 등임. XML은 개발자가 태그를 직접 정의하여 사용할 수 있는 장점이 있음. 
 

In [6]:
def get_link_from_news_title(page_num, URL):
    for i in range(page_num): #전체 페이지가 들어온다.
        current_page_num = 1 + i #페이지는 1부터
        position = TARGET_URL.index('page=') 
        #position = URL.index('page=') 이거를 넣는게 논리적으로 조금 더 자연스럽습니다. 
        URL_with_page_num = TARGET_URL[:position+5] + str(current_page_num) + TARGET_URL[position+5:] 
        #URL_with_page_num = URL[:position+5] + str(current_page_num) + URL[position+5:] 위와 동일
        source_code_form_url = urllib.request.urlopen(URL_with_page_num)
        soup = BeautifulSoup(source_code_form_url, 'lxml', from_encoding='utf-8')
        for title in soup.select('h2.headline.mg'):
            url_link = title.select('a')
            # TITLE_OF_ARTICLE.append(url_link[0].string)
            article_URL = url_link[0]['href'] 
            get_text(article_URL)

**get_text** :  기사 본문 내용 긁어오기 (위 함수 get_link_from_news_title에서 기사 본문 주소 받아서 사용되는 함수) 
* source_code_from_url : 위 함수에서 받은 URL에 접속 요청을 한다.
* soup : 데이터를 가져오기 위한 BeautifulSoup Library
* soup.select('#article_title)을 통해서 제목을 가리킴. 그리고 다음 열에서 title_link의 0번째 문자열 중 string을 가져와 appendix하는 방식임.
*   date_url : date가 위치하는 HTML을 가리킨다.
*   date_of_article = date가 위치하는 HTML에 다양한 정보가 있으므로 date의 정확한 위치를 가리키는 변수
*   DATE_OF_ARTICLE.append : 날짜를 리스트에 넣는다.
* content : 기사의 본문이 위치한 곳을 가리키는 변수
* string_item : html의 tag를 제외한 텍스트를 가져온다.
* CONTENT_OF_ARTICLE : 기사의 내용을 리스트에 넣는다.
* 한가지 명심할 점! 위 노트의 get_text(article_URL)에서의 get_text와 아래 노트의 def get_text(URL)에서의 get_text는 다른 것임! 한 노트에서 연결된 코드 내에서 '지역변수'(*위에서 설명)는 해당 코드의 영역을 벗어나면 아무런 의미가 없음. 그냥 쓰고 버리는 것으로 보면 됨.


In [7]:
def get_text(URL):
    source_code_form_url = urllib.request.urlopen(URL)
    soup = BeautifulSoup(source_code_form_url, 'lxml', from_encoding='utf-8')
  
    title_link = soup.select('#article_title') # 타이틀 크롤링, 리스트에 넣어줌
    TITLE_OF_ARTICLE.append(title_link[0].string)

    date_url = soup.select('div.byline') # 날짜 크롤링, 리스트에 넣어줌
    date_of_article = date_url[0].select('em')
    DATE_OF_ARTICLE.append(date_of_article[1].string)
  
    content = soup.select('div.article_body') # 기사본문 크롤링, 리스트에 넣어줌
    for item in content:
        string_item = str(item.find_all(text=True))
        CONTENT_OF_ARTICLE.append(string_item)

In [1]:
def get_total_num_of_article(URL):
    source_code_from_url = urllib.request.urlopen(URL)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    total_num_of_article = soup.select('span.total_number')
    num = total_num_of_article[0].string
    position = num.index('/') # 1-72/725건이라면, 725건만 가지고 검색하겠다. 나머지는 버리겠다는 의미
    num = num[position:]  # position인 '/' 오른쪽에 있는 부분만 남김 
    num = re.sub('[^0-9]', '', str(num)) # 0-9사이 숫자아닌것 모두 제거. 725 만 남음
    num = int(num)
    if(num % 10 >= 1): # 725을 10으로 나눴을때 나머지가 1보다 크거나 같으면
        page_num = int((num / 10) + 1) #그럴경우 10으로 나눠 1을 더함. 73가 됨
    else: 
        page_num = int(num / 10) # 그렇지 않을 경우는 그냥 10으로 나눠, 71이될거임
    return page_num

page_num = get_total_num_of_article(TARGET_URL)

In [31]:
get_link_from_news_title(2, TARGET_URL)
#get_link_from_news_title(page_num, TARGET_URL)

In [32]:
df = pd.DataFrame(list(zip(TITLE_OF_ARTICLE, CONTENT_OF_ARTICLE, DATE_OF_ARTICLE)), columns =['Title', 'Content', 'Date']) 

In [33]:
df.head()

Title  ...                 Date
0   변희수 사망 전날…軍, 법원에 '성기훼손은 자해' 주장했다     ...  입력 2021.03.05 21:13
1   추미애 "늦었지만 변희수 애도, 국회 '차별금지법' 나서라"    ...  입력 2021.03.05 19:37
2   왕의 명령에 한국 왔던 중동 청년들…화성탐사선 쏘아올렸다      ...  입력 2021.03.05 18:00
3   왕의 명령에 한국 왔던 중동 청년들…화성탐사선 쏘아올렸다      ...  입력 2021.03.05 18:00
4   '성전환 군인' 변희수 애도한 종교계 "그 죽음은 사회적 타살"  ...  입력 2021.03.05 17:52

[5 rows x 3 columns]

# Data Cleaning

Date cleaning & divide date to year, month, day  
Date: 2020-05-22를 Year: 2020, Month: 05, Day: 22로 분리 

In [34]:
df['Date'] = df['Date'].str.replace('[^0-9]', '', regex=True)
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d%H%M')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [35]:
df = df.drop(['Date'], axis=1)

본문  
이메일 제거
* 정규 표현식(https://docs.python.org/3.7/howto/regex.html)
정규 표현식에서 사용하는 메타 문자(meta characters)에는 다음과 같은 것이 있다.
※ 메타 문자란 원래 그 문자가 가진 뜻이 아닌 특별한 용도로 사용하는 문자를 말한다.
. ^ $ * + ? { } [ ] \ | ( )
정규 표현식에 위 메타 문자를 사용하면 특별한 의미를 갖게 된다.

* 문자 클래스 [ ]
문자 클래스를 만드는 메타 문자인 [ ] 사이에는 어떤 문자도 들어갈 수 있다.
[ ] 안의 두 문자 사이에 하이픈(-)을 사용하면 두 문자 사이의 범위(From - To)를 의미한다.

* Dot(.)
정규 표현식의 Dot(.) 메타 문자는 줄바꿈 문자인 \n을 제외한 모든 문자와 매치됨을 의미한다.
a.b = "a + 모든문자 + b"
※ 문자 클래스([]) 내에 Dot(.) 메타 문자가 사용된다면 이것은 "모든 문자"라는 의미가 아닌 문자 . 그대로를 의미한다.

* 반복 (+)
반복을 나타내는 또 다른 메타 문자로 +가 있다. +는 최소 1번 이상 반복될 때 사용한다. 즉 *가 반복 횟수 0부터라면 +는 반복 횟수 1부터인 것이다.

[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9]+

jkoo@skku.edu

id : [a-zA-Z0-9_.+-]+ <-- jkoo

* [ : 문자 클래스
* a-z : 모든 소문자
* A-Z : 모든 대문자
* 0-9 : 모든 숫자
* _.+- : 이메일에서 사용될수 있는 특수문자들
* ] : 문자 클래스
* + : 앞의 문자 클래스안에 있는 문자들이 한번 혹은 반복됨을 알린다.
이메일 아이디는 예외처리를 하지 않았을 때 특수 문자 !#$%&'*+-/=?^_`{|}~ 등이 포함될 수 있는데 일반적으로 자주 사용되는 _.+-를 포함하였습니다.

domain name @[a-zA-Z0-9-]+ <-- skku

domain \.[a-zA-Z0-9]+ <-- edu

* 닷의 경우 \ (역슬래쉬)를 붙여주지 않으면 메타문자로 인식하기 때문에 앞에 \가 있습니다.

In [36]:
df['Content'] = df['Content'].str.replace(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)" , '')

본문  
불필요한 기호 및 문자 제거

In [37]:
df['Content'] = df['Content'].str.replace('[', '')
df['Content'] = df['Content'].str.replace(']', '')
df['Content'] = df['Content'].str.replace("'", "")  
df['Content'] = df['Content'].str.replace('"', '')
df['Content'] = df['Content'].str.replace(',', '')
df['Content'] = df['Content'].str.replace(r'\\n', '') 
df['Content'] = df['Content'].str.replace(r'\\r', '')
df['Content'] = df['Content'].str.replace(r'\\xa0', '')
df['Content'] = df['Content'].str.replace('‘', '')
df['Content'] = df['Content'].str.replace('’', '')
df['Content'] = df['Content'].str.replace('“', '')
df['Content'] = df['Content'].str.replace('”', '')
df['Content'] = df['Content'].str.replace('.', '')
df['Content'] = df['Content'].str.replace('//', '')
df['Content'] = df['Content'].str.replace('ㆍ', '')
df['Content'] = df['Content'].str.replace('  ', '')
df['Content'] = df['Content'].str.replace('·', '')
df['Content'] = df['Content'].str.replace('…', '')
df['Content'] = df['Content'].str.replace('〈br〉', '')
df['Content'] = df['Content'].str.replace('「', '')
df['Content'] = df['Content'].str.replace('」', '')
df['Content'] = df['Content'].str.replace(r'\\', '')
df['Content'] = df['Content'].str.replace('아티클 공통 : DA 250', '') 
df['Content'] = df['Content'].str.replace('아티클 공통 : 관련기사', '')
df['Content'] = df['Content'].str.lstrip() #공백제거

본문  
시진핑(習近平) -> 시진핑   
중국공산당[CCP·Chinese Communist Party] -> 중국공산당     
괄호 및 대괄호안에 한문, 영어등 삭제

In [38]:
df['Content'] = df['Content'].str.replace(r'\(([^)]+)\)','')
#df['Content'] = df['Content'].str.replace(r'〈([^〉]+)〉','')

제목  
불필요한 기호 및 문자 제거

In [39]:
df['Title'] = df['Title'].str.replace(r'\[([^)]+)\]','') # [단독]등등
df['Title'] = df['Title'].str.replace('‘', '')
df['Title'] = df['Title'].str.replace('’', '')
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].str.replace('[', '')
df['Title'] = df['Title'].str.replace(']', '')
df['Title'] = df['Title'].str.replace('…', '')
df['Title'] = df['Title'].str.replace('-', '')
df['Title'] = df['Title'].str.replace('·', '')
df['Title'] = df['Title'].str.replace('“', '')
df['Title'] = df['Title'].str.replace('”', '')
df['Title'] = df['Title'].str.replace('"', '')
df['Title'] = df['Title'].str.replace('···', '')
df['Title'] = df['Title'].str.replace("'", '')
df['Title'] = df['Title'].str.replace('?', '')

In [40]:
df.tail()

,Title,Content,Year,Month,Day
15,유엔 미얀마 살인 멈춰야쿠데타 이후 최소 54명 숨져,4일 미얀마 만달레이에서 시민들이 쿠데타 반대 시위 중 숨진 희생자를 추모 하고 있다 EPA=연합뉴스 미얀마에서 지난달 1일 발생한 쿠데타 이후 군경의 총격으로 최소 54명이 숨졌고 1700명 이상이 구금됐다고 유엔이 4일 밝혔다미셸 바첼레트 유엔 인권최고대표는 이날 발표한 성명에서 미얀마 군경의 총격에 희생된 사람은 최소 54명이라며 미얀마 군부에 살인을 중단하라고 촉구했다바첼레트 대표는 사망자 54명 가운데 최소 30명이 전날 양곤과 만달레이 사가잉 매그웨이 몬에서 18명은 지난달 28일 숨진 것으로 파악됐다고 밝혔다그는 또 쿠데타 이후 1700명 이상이 구금됐고 최근에는 언론인도 29명 이상 체포됐다고 말했다 그러면서 미얀마 군부는 시위대에 대한 잔인한 탄압과 살인을 멈춰야 한다고 강조했다전날 미얀마에서는 군부가 실탄을 동원해 시위대를 강경 진압하면서 전역에서 사망자가 속출했다 AFP통신에 따르면 슈래너 버기너 유엔 미얀마 특사는 기자회견을 통해 오늘은 2월 1일 쿠데타 발생 이후 가장 많은 피를 흘린 날이라며 이제 쿠데타 이후 총 사망자가 50명을 넘었다고 말했다 그러면서 미얀마에서 진짜 전쟁이 벌어질 수도 있다고 우려했다고 로이터통신은 전했다 AP통신은 미얀마 현지 데이터 전문가를 인용해 3일 하루 만에 최소 34명이 사망했다고 보도했다미얀마 군부는 지난해 11월 총선에서 심각한 부정이 발생했음에도 문민정부가 이를 제대로 조사하지 않았다는 이유를 들며 지난달 1일 쿠데타를 일으켜 정권을 잡았다이후 미얀마 국민은 최대 도시 양곤 등 곳곳에서 쿠데타를 규탄하는 대규모 시위를 한 달여간 이어가고 있다정혜정 기자,2021,3,4
16,멈춰선 기갑의 돌파력성전환 하사의 외로웠던 투쟁,4일 새벽 군인권센터에는기갑의 돌파력으로 군의 소수자 차별을 없애버리겠다던 전차조종수 변 하사를 기억한다는 글이 올라왔다 전날 자택에서 숨진 채 발견된 변희수 전 하사를 추모하는 글이다 고인은 군 복무 중 성전환 수술을 했다는 이유로 지난해 1월 육군으로부터 강제 전역 처분을 받았다성 정체성과 상관없이 나라를 지키고 싶다 기회를 달라며 국가를 상대로 소송을 낸 기갑의 돌파력은 멈춰섰다 변희수 전 하사가 지난해 1월 22일 기자회견에서 사적 국외여행 허가서를 보여주고 있다 뉴시스 군인이라는 자부심 높았던 사람국가인권위원회는 이날 뿌리 깊은 차별과 혐오에 맞서다 사망한 그의 노력은 앞으로도 영원히 기억될 것이라고 애도했다 변 전 하사의 행정소송을 돕던 김보라미 변호사는 군인이라는 자부심이 높은 사람이었다고 회고했다군인권센터에 따르면 4일 마련된 청주성모병원 빈소에는 오전부터 변 전 하사를 추모하는 조문객들의 발길이 이어졌다 부고를 접한 네티즌들도 떠난 그곳에서는 차별 없이 편안하길 바란다며 명복을 빌었다 반면 고인의 죽음은 안타깝지만 그의 행동을 지지하지는 않는다는 댓글도 적지 않았다4일 경찰 등에 따르면 변 전 하사는 3일 오후 5시 50분쯤 자택에서 숨진 상태로 소방대에 발견됐다 변 전 하사를 돕던 청주시 상당구 정신건강센터가 지난달 28일부터 그와 연락이 닿지 않아 신고를 했다 그가 3개월 전 극단적 선택을 시도했고 이후 정신과 치료를 받았던 사실도 뒤늦게 알려졌다 성전환 수술을 한 뒤 강제 전역한 변희수 전 하사의 사망 소식이 전해진 4일 서울 여의도 국회 정의당 대표실 앞에 변 전 하사의 추모공간이 마련돼 있다 뉴스1 군 성 전환 수술을 심신장애로 판단변 전 하사의 외로운 투쟁은 2년 여 전 공론화됐다 2017년 남성으로 입대한 변 전 하사는 기갑병과 전차 승무 특기로 임관해 경기 북부 지역의 한 육군부대에 복무했다 성 정체성을 고민하던그는 2019년 11월 휴가 중 태국에서 성전환 수술을 받고 돌아온 뒤 계속 복무를 희망했다남성 군인의 성 전환과 계속 복무 요구 이는 성 소수자 문제를 외면하는 데 익숙한 한국 사회에 대한 도발이었다 육군 측은 변 전 하사를 심신장애 3급으로 판단하고 지난해 1월 전역 결정을 내렸다그의 강제 전역은 찬반 논란을 불러왔다 육군은 성기와 고환이 제거됐기 때문에 군인의 임무를 수행할 수 없다고 판단했다 군의 특수성을 고려해야 한다 함께 복무하는 군인들에게 불편하다는 여론도 이어졌다 변 전 하사와 군인권센터는 수술의 결과는 기능장애나 신체 훼손에 해당하지 않는다 전역 처분은 법률적 근거가 없다고 맞섰다 11일 오전 서울 종로구 참여연대에서 열린 전역 처분 취소 행정소송 제기 기자회견에 생각에 잠겨있는 변 전 하사 뉴스1 전역 처분 취소 소송 계속될까변 전 하사가 지난해 8월 육군참모총장을 상대로 낸 전역 처분 취소 소송은 다음달 15일 첫 변론 기일을 앞둔 상태였다 법조계에서는 그의 죽음으로 전역 취소 소송이 불투명해졌다는 의견이 우세하다 익명을 요구한 서초동의 한 변호사는 전역 취소 처분이 뒤집히더라도 복직을 할 당사자가 사라져 버린 것이라며 행정소송이 각하될 확률이 높다고 말했다박찬성 변호사는 그동안 받지 못했던 급여 등을 법률상 이익으로 볼 수 있다 쉽지 않겠지만 원칙적으로는 유족이 소송을 계속 진행할 수 있다고 주장했다 지난해 12월 서울행정법원은 전역 후 사망한 육군 대위의 전역 처분 취소 소송에서 국방부의 전역 취소에 중대한 하자가 있다고 판단하기도 했다 이 소송은 대위의 아내가 제기했다변 전 하사의 소송대리인단 관계자는 유족분들의 입장이 가장 중요하지 않겠냐 지금은 장례에 집중하겠다고 했다편광현 기자,2021,3,4
17,윤석열 사의 수용 45분뒤 신현수 교체文의 정면대응,문재인 대통령은 4일 윤석열 검찰총장의 사의를 즉각 수용했다 동시에 윤 총장과의 사실상 유일한 소통 채널이던 신현수 청와대 민정수석의 사표를 수리하고 후임 민정수석에 김진국 감사원 감사위원을 임명했다 윤석열 검찰총장이 4일 오후 서울 서초구 대검찰청 현관에서 총장직 사퇴 의사를 발표하고 있다 윤 총장은 최근 중대범죄수사청 설치 문제를 두고 여권과 날카롭게 대립해 왔다 뉴스1 정만호 청와대 국민소통수석은 이날 오후 3시 15분 문 대통령이 윤석열 검찰총장의 사의를 수용했다는 짧은 공식 입장을 밝혔다 윤 총장이 오후 2시 사의를 표명한지 1시간여만에 나온 공식 브리핑이었다청와대 고위관계자는 사표 수리 과정에 대해 법무부에 윤 총장이 제출한 사표가 접수됐고 사표 수리와 관련해서는 앞으로 행정절차가 진행될 것이라고 설명했다 그러면서 윤 총장의 후임 임명도 법에 정해진 관련 절차를 밟아서 진행될 것이라고 덧붙였다정 수석은 45분 뒤인 오후 4시 재차 브리핑을 열어 신 수석의 후임에 김진국 감사위원을 임명했다고 밝혔다 검찰총장의 사퇴에서 청와대 민정수석 교체까지 진행되는데 불과 불과 2시간 밖에 걸리지 않았다 사실상 청와대가 윤 총장 사퇴에 대비하고 있었음을 시사하는 대목이다 4일 임명된 김진국 대통령비서실 민정수석비서관이 청와대 춘추관에서 인사말을 하기위해 연단으로 올라서며 인사말을 마친 전임 신현수 수석과 교차하고 있다 연합뉴스 청와대 사정에 밝은 여권 관계자는 거취가 애매하던 신현수 수석까지 신속히 교체한 것은 윤 총장의 일방적 사퇴 선언에 따른 문 대통령의 정면대응 차원으로 이해된다고 평가했다 그는 결국 애초에 신 수석을 기용한 의도가 윤 총장을 관리하기 위한 것임이 확인된 셈이라며 윤 총장이 떠나버렸기 때문에 신 수석의 용도 역시 공식 폐기됐음을 선언한 것이라고 덧붙였다 윤석열 검찰총장이 4일 오후 서울 서초구 대검찰청 현관에서 총장직 사퇴 의사를 발표하고 있다 윤 총장은 최근 중대범죄수사청 설치 문제를 두고 여권과 날카롭게 대립해 왔다 뉴스1문 대통령은 신 수석의 후임으로 또 다시 비검찰 출신을 임명했다 신임 김진국 수석은 인권 변호사 출신으로 노무현 정부 때 문 대통령이 이끌던 민정수석실의 법무비서관으로 근무한 경험이 있다 문재인 정부 들어서는 감사원 감사위원을 맡아 왔다문재인 정부에서 임명된 5명의 민정수석 중 감사원 출신은 김조원김종호 전 수석에 이어 이번이 세번째다여권에선 비검찰 출신 민정수석이 재등장한 배경에 대해 문 대통령이 또 다시 검찰을 개혁 대상으로 공식화하겠다는 뜻을 나타낸 것이란 평가가 나왔다 그런 차

In [41]:
df.to_csv("df.csv",index=True, encoding='utf-8-sig')